In [63]:
%reset

In [64]:
import pandas as pd
import tensorflow as tf
df_rus = pd.read_excel("ready_rus.xlsx")
df_rus.head(2)


,review,sentiment
0,В 1974 году подросток Марта Моксли (Мэгги Грей...,1
1,Хорошо... итак... Мне очень нравится Крис Крис...,0


In [65]:
target = df_rus.pop('sentiment')

In [66]:
df_rus.head(2)

,review
0,В 1974 году подросток Марта Моксли (Мэгги Грей...
1,Хорошо... итак... Мне очень нравится Крис Крис...


In [67]:
ds_raw = tf.data.Dataset.from_tensor_slices((df_rus.values,target.values))
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50],ex[1])

(b'\xd0\x92 1974 \xd0\xb3\xd0\xbe\xd0\xb4\xd1\x83 \xd0\xbf\xd0\xbe\xd0\xb4\xd1'
 b'\x80\xd0\xbe\xd1\x81\xd1\x82\xd0\xbe\xd0\xba \xd0\x9c\xd0\xb0'
 b'\xd1\x80\xd1\x82\xd0\xb0 \xd0\x9c\xd0') 1
(b'\xd0\xa5\xd0\xbe\xd1\x80\xd0\xbe\xd1\x88\xd0\xbe... \xd0\xb8\xd1\x82'
 b'\xd0\xb0\xd0\xba... \xd0\x9c\xd0\xbd\xd0\xb5 \xd0\xbe\xd1\x87\xd0'
 b'\xb5\xd0\xbd\xd1\x8c \xd0\xbd\xd1\x80') 0
(b'*** \xd0\xa1\xd0\x9f\xd0\x9e\xd0\x99\xd0\x9b\xd0\x95\xd0\xa0 *** \xd0'
 b'\x9d\xd0\xb5 \xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd0\xb9\xd1\x82\xd0\xb5 \xd1'
 b'\x8d\xd1\x82\xd0\xbe,') 0


In [68]:
"""Делим датасет на тренировочный, тестовый и валидационный"""
tf.random.set_seed(1)
ds_raw = ds_raw.shuffle(120,reshuffle_each_iteration=False)
ds_raw_test = ds_raw.take(35)
ds_raw_train_valid = ds_raw.skip(30)
ds_raw_train = ds_raw_train_valid.take(60)
ds_raw_valid = ds_raw_train_valid.skip(20)

In [69]:
for i in ds_raw:
    print(i[0].numpy())

[b'\xd0\xaf \xd0\xb2\xd0\xbe\xd0\xb7\xd0\xbb\xd0\xb0\xd0\xb3\xd0\xb0\xd0\xbb \xd0\xbd\xd0\xb0 \xd1\x8d\xd1\x82\xd0\xbe\xd1\x82 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc \xd0\xb1\xd0\xbe\xd0\xbb\xd1\x8c\xd1\x88\xd0\xb8\xd0\xb5 \xd0\xbd\xd0\xb0\xd0\xb4\xd0\xb5\xd0\xb6\xd0\xb4\xd1\x8b, \xd0\xbf\xd0\xbe\xd0\xba\xd0\xb0 \xd0\xbe\xd0\xbd\xd0\xb8 \xd0\xbd\xd0\xb5 \xd1\x81\xd0\xbc\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xbb\xd0\xb8 \xd0\xbd\xd0\xb0\xd0\xb7\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xbd\xd0\xb0 "\xd0\x9f\xd0\xb0\xd1\x81\xd1\x82\xd1\x83\xd1\x85: \xd0\xbf\xd0\xbe\xd0\xb3\xd1\x80\xd0\xb0\xd0\xbd\xd0\xb8\xd1\x87\xd0\xbd\xd1\x8b\xd0\xb9 \xd0\xbf\xd0\xb0\xd1\x82\xd1\x80\xd1\x83\xd0\xbb\xd1\x8c", \xd1\x81\xd0\xb0\xd0\xbc\xd0\xbe\xd0\xb5 \xd0\xb4\xd1\x83\xd1\x80\xd0\xb0\xd1\x86\xd0\xba\xd0\xbe\xd0\xb5 \xd0\xbd\xd0\xb0\xd0\xb7\xd0\xb2\xd0\xb0\xd0\xbd\xd0\xb8\xd0\xb5 \xd1\x84\xd0\xb8\xd0\xbb\xd1\x8c\xd0\xbc\xd0\xb0 \xd0\xb7\xd0\xb0 \xd0\xb2\xd1\x81\xd1\x8e \xd0\xb8\xd1\x81\xd1\x82\xd0\xbe\xd1\x80\x

In [70]:

from collections import Counter
import tensorflow_datasets as tfds
tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()
for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy()[0]) #делит текст по словам
    token_counts.update(tokens) #считает уникальные слова
token_counts

Counter({'Никакого': 1,
         'смеха': 2,
         'вообще': 4,
         'Да': 3,
         'я': 113,
         'наблюдал': 1,
         'за': 57,
         'всем': 7,
         'этим': 10,
         'крушением': 1,
         'поезда': 2,
         'но': 96,
         'только': 29,
         'для': 62,
         'того': 30,
         'чтобы': 60,
         'потом': 4,
         'не': 199,
         'задаваться': 1,
         'вопросом': 1,
         'пришел': 1,
         'ли': 12,
         'Клиз': 1,
         'в': 341,
         'себя': 15,
         'последней': 2,
         'части': 3,
         'Нет': 3,
         'он': 89,
         'этого': 32,
         'делал': 1,
         'br': 342,
         'Это': 49,
         'может': 16,
         'быть': 30,
         'исторически': 2,
         'интересно': 5,
         'вас': 11,
         'молодежи': 1,
         'увидеть': 5,
         'что': 227,
         'британский': 1,
         'юмор': 2,
         'включал': 1,
         'черные': 1,
         'шутки': 4,
      

In [71]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
example_str = "Это пример!"
encoder.encode(example_str)


[33, 5415]

In [72]:
def encode(text_tensor,label):
    """Декодирует текстовый тензор в лист индексов слов"""
    text = text_tensor.numpy()[0]
    encoded_text = encoder.encode(text)
    return encoded_text,label

def encode_map_fn(text,label):
    """Завертывает питоновскую функцию в tensorflow"""
    return tf.py_function(encode,inp=[text,label],Tout=(tf.int64,tf.int64))

ds_train = ds_raw_train.map(encode_map_fn)
ds_valid = ds_raw_valid.map(encode_map_fn)
ds_test = ds_raw_test.map(encode_map_fn)



In [ ]:
"""Размер у каждого набора разный - для того, чтобы привести к единому размеру,
используем batch_padded который заполняет пропуски нулями"""
ds_subset = ds_train.take(8)
for ex in ds_subset:
    print('не забатченный',ex[0].shape)
ds_batched = ds_subset.padded_batch(4,padded_shapes=([-1],[]))
for batch in ds_batched:
    print('забатченный',batch[0].shape)

In [73]:
train_data = ds_train.padded_batch(32,padded_shapes=([-1],[]))
valid_data = ds_valid.padded_batch(32,padded_shapes=([-1],[]))
test_data = ds_test.padded_batch(32,padded_shapes=([-1],[]))

In [75]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Embedding,SimpleRNN,Bidirectional,LSTM
embedding_dim = 20
vocab_size = len(token_counts)+2
model = Sequential()
model.add(Embedding(
    input_dim=vocab_size, # размер целочисленных уникальных значений,
    output_dim=embedding_dim, #размер признаков вложения
    name = 'embedded-layer'
 ))
model.add(Bidirectional( #для того, чтобы рекурентный слой шёл туда и обратно по псоледовательности
    LSTM(
        64,
        name='lstm-layers'
    ),
    name = 'bidir-lstm'
))
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedded-layer (Embedding)  (None, None, 20)          108320    
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 160,161
Trainable params: 160,161
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_data,validation_data=valid_data,epochs=10)



Epoch 1/10
2/2 [==============================] - 4s 1s/step - loss: 0.6947 - accuracy: 0.4667 - val_loss: 0.6919 - val_accuracy: 0.5224
Epoch 2/10
2/2 [==============================] - 1s 557ms/step - loss: 0.6927 - accuracy: 0.4833 - val_loss: 0.6917 - val_accuracy: 0.5373
Epoch 3/10
2/2 [==============================] - 1s 481ms/step - loss: 0.6913 - accuracy: 0.4833 - val_loss: 0.6914 - val_accuracy: 0.5672
Epoch 4/10
2/2 [==============================] - 1s 490ms/step - loss: 0.6897 - accuracy: 0.7500 - val_loss: 0.6908 - val_accuracy: 0.7164

C:\Users\karim\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


In [78]:
test_res = model.evaluate(test_data)
test_res[0]

2/2 [==============================] - 0s 18ms/step - loss: 0.6737 - accuracy: 0.7143


0.6737229228019714

In [79]:
pd.read_csv("books/1268.csv")


ParserError: Error tokenizing data. C error: Expected 2 fields in line 52, saw 4
